In [2]:
#!conda install -c anaconda beautifulsoup4 --yes
print("Beautiful soup installed")

WARNING conda.base.context:use_only_tar_bz2(632): Conda is constrained to only using the old .tar.bz2 file format because you have conda-build installed, and it is <3.18.3.  Update or remove conda-build to get smaller downloads and faster extractions.
Solving environment: / 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/linux-64::anaconda==5.3.1=py37_0
  - defaults/linux-64::astropy==3.0.4=py37h14c3975_0
  - defaults/linux-64::bkcharts==0.2=py37_0
  - defaults/linux-64::blaze==0.11.3=py37_0
  - defaults/linux-64::bokeh==0.13.0=py37_0
  - defaults/linux-64::bottleneck==1.2.1=py37h035aef0_1
  - defaults/linux-64::dask==0.19.1=py37_0
  - defaults/linux-64::datashape==0.5.4=py37_1
  - defaults/linux-64::mkl-service==1.1.2=py37h90e4bf4_5
  - defaults/linux-64::numba==0.39.0=py37h04863e7_0
  - defaults/linux-64::numexpr==2.6.8=py37hd89afb7_0
  - defaults/linux-64::odo==0.5.1=py37_0
  - defaults/lin

Download the file and store it in Canada_zipcode.html

In [1]:
!wget -q -O 'canada_zipcode.html' https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M


Code to parse the table from HTML and store it in Dataframe

In [1]:
from bs4 import BeautifulSoup
import requests as req    
import pandas as pd
contents = req.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
res=[]
soup = BeautifulSoup(contents.text, 'lxml')
table = soup.find('table')
table_rows = table.find_all('tr')
data_row=pd.DataFrame()
for tr in table_rows:
    td = tr.find_all('td')
    row = [i.text for i in td]
    if(len(row)>0):
        res.append(row)     
df_canada=pd.DataFrame(res,columns=['Postal Code','Borough','Neighborhood'])
df_canada['Neighborhood']=df_canada['Neighborhood'].replace(regex='\n',value="")
df_canada.head()


,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


Droping the rows with  Borough as Not Assigned

In [2]:
df_canada.drop(df_canada[df_canada['Borough'] =='Not assigned'].index, inplace = True)
df_canada.shape

(211, 3)

Assign the value of Borough to Neighborhood if the value is Not Assigned

In [3]:
import numpy as np
df_canada['Neighborhood']=np.where(df_canada['Neighborhood']=='Not assigned',df_canada['Borough'],df_canada['Neighborhood'])
df_canada[['Borough','Neighborhood']].head(10)

,Borough,Neighborhood
2,North York,Parkwoods
3,North York,Victoria Village
4,Downtown Toronto,Harbourfront
5,Downtown Toronto,Regent Park
6,North York,Lawrence Heights
7,North York,Lawrence Manor
8,Queen's Park,Queen's Park
10,Etobicoke,Islington Avenue
11,Scarborough,Rouge
12,Scarborough,Malvern


In [4]:
import numpy as np
df_canada['Neighborhood']=np.where(df_canada['Neighborhood']=='Not assigned',df_canada['Borough'],df_canada['Neighborhood'])
df_canada_update=df_canada
df_canada_update.head()

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


Groupby Neighbourhood

In [6]:
df_canada=df_canada_update.groupby(['Postal Code','Borough'])['Neighborhood'].apply(','.join).reset_index()
df_canada.head()

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
